In [11]:
pip install finta

Note: you may need to restart the kernel to use updated packages.


In [12]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import random
from finta.finta import TA
import matplotlib.pyplot as plt
from os.path import isfile, join
import pickle
import concurrent.futures  as cf
import copy
from time import perf_counter
from multiprocessing import Pool, cpu_count
import gc

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
history = []
cdir = os.getcwd()
path = os.path.join(cdir, 'history', 'minute')


# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [13]:
path = "../input/btcmin"
os.mkdir(os.path.join(cdir,'results'))
os.mkdir(os.path.join(cdir,'pickle'))

for dirname, _, filenames in os.walk(path):
    for filename in filenames:
        df : pd.DataFrame = pd.read_csv(os.path.join(dirname, filename), sep = ',', decimal='.', date_parser=True)
        criptoName = filename.replace('Binance_', '').replace('USDT_minute.csv', '')
        df.drop(['unix', 'tradecount', 'symbol', f'Volume {criptoName}'], axis=1, inplace=True)   
        df['date'] = pd.to_datetime(df['date'])
        #df = df[df['date']>'2022-04-10 00:00:00']
        df = df.sort_values(by='date', ascending=True)
 
        df['MINUTE'] = df['date'].dt.minute
        times = [0,10,20,30,40,50] #0,5,10,15,20,25,30,35,40,45,50,55
        df = df[df['MINUTE'].isin(times)]
        del df['MINUTE']
        df = df.set_index(pd.DatetimeIndex(df['date']))

        df.drop('date', axis=1, inplace=True)
        df = df.rename(columns={'Volume USDT': 'volume'})
        df = df[~df.index.duplicated()]
        df.reset_index(drop=True, inplace=True)
        # RSI
        df['RSI'] = TA.RSI(df).round(0).fillna(0).astype(int)

        #MACD
        macd = TA.MACD(df)
        df['MACDS1'] = macd['MACD']
        df['MACDS2'] = macd['SIGNAL']


        #EMAS
        df['EMA18'] = TA.EMA(df, 18)
        df['EMA36'] = TA.EMA(df, 36)
        df['EMA140'] =TA.EMA(df, 140)
        df['EMA200'] =TA.EMA(df, 200)


        #BBANS
        BBANS = TA.BBANDS(df)
        df['BB_UPPER'] = BBANS['BB_UPPER']
        df['BB_LOWER'] = BBANS['BB_LOWER']
        df['moedas'] = 0
        df['saldo'] = 0
        df['lucro'] = 0
        df['HBS'] = 'H'
        df = df.fillna(0)
        history.append([filename, df])

FileExistsError: [Errno 17] File exists: '/kaggle/working/results'

In [ ]:

class StockTradingEnv():
    def __init__(self, df):
        self.saldo = 1000
        self.lucro=0
        self.position = 0
        self.moedas=0
        self.df: pd.DataFrame = df
        self.observation_space = df.shape[1]-1
        self.action_space = 3

        
    def _next_observation(self):
        self.position+=1
        #obs = 0
        if self.moedas==0: 
            self.df.at[self.position, 'saldo'] = round(self.saldo,2)
            self.df.at[self.position, 'lucro'] = round(self.lucro, 2)
            self.df.at[self.position, 'moedas'] = round(self.moedas, 6)
        else:
            self.df.at[self.position, 'saldo'] =round(self.df.iloc[self.position].close * self.moedas,2)
            custo =  0#self.saldo * 0.001
            self.lucro=(self.df.iloc[self.position].saldo - 1000  ) -custo
            self.df.at[self.position, 'lucro'] = round(self.lucro, 2)
            self.df.at[self.position, 'moedas'] = round(self.moedas, 6)
            #try:
        obs = self.df.iloc[self.position][:self.df.shape[1]-1].fillna(0).to_numpy()
            #except:
            #obs = self.df.iloc[self.position].fillna(0).to_numpy()

        return obs

    def _take_action(self, action):
        
        #buy
        if action ==0:
            if self.df.iloc[self.position].moedas == 0:
                custo = 0 #self.df.iloc[self.position].saldo * 0.001
                self.moedas = self.df.iloc[self.position].saldo / self.df.iloc[self.position].close

                self.lucro-=custo
                self.df.at[self.position, 'moedas'] = round(self.moedas, 6)
                self.df.at[self.position, 'lucro'] = round(self.lucro, 2)
                self.df.at[self.position, 'HBS'] = 'B'
                #print(f'Comprei {round(round(self.moedas, 6),2)} moedas, por {self.df.iloc[self.position].close}')
            
        
        if action == 2:
            if self.df.iloc[self.position].moedas > 0:
                self.saldo = self.df.iloc[self.position].moedas * self.df.iloc[self.position].close 
                custo =  0 #self.saldo * 0.001
                self.lucro = (self.saldo - 1000  ) -custo
                #print(f'Vendi {round(round(self.moedas, 6),2)} moedas, por {self.df.iloc[self.position].close}, lucro de {round(round(self.lucro, 2),2)}')
                self.moedas = 0
                self.df.at[self.position, 'saldo'] = round(self.saldo , 2) 
                self.df.at[self.position, 'lucro'] = round(self.lucro, 2) 
                self.df.at[self.position, 'moedas'] = 0
                self.df.at[self.position, 'HBS'] = 'S'
            
                
        return self.df.iloc[self.position].lucro

    def step(self, action):
        #print(action)
        try:
            l = list(action).index(np.max(action))
        except:
            self.df[:10000].to_csv('result_FAIL_.csv', sep=';', decimal=',')

        reward = self._take_action(l)
        done = False
        obs = self._next_observation()

        return obs, reward, done, self.df.at[self.position, 'saldo']

    def reset(self, locked=False):
        self.saldo = 1000
        self.lucro=0
        self.position = 0
        self.moedas=0
        if locked == False:
            rnd_history = random.randrange(0, len(history)-1)
            self.df = history[rnd_history][1]
            self.position = random.randrange(150, len(self.df)-400)
        else:
            for i in range(0, len(history)-1):
                if 'BTC' in history[i][0]:
                    self.df = history[i][1]
        self.df['moedas'] = 0
        self.df['saldo'] = 0
        self.df['lucro'] = 0
        self.df['HBS'] = 'H'
        return self._next_observation()

    def render(self, mode='human', close=False):
        # Render the environment to the screen
        pass

In [ ]:
class Hp():
    def __init__(self):
        self.nb_steps = 1000000
        self.episode_length = 500
        self.episode_length2 = 10000
        self.learning_rate = 0.001
        self.nb_directions = 16
        self.nb_best_directions = 16
        assert self.nb_best_directions <= self.nb_directions
        self.noise = 0.03
        self.seed = 1
        self.env_name = 'HalfCheetahBulletEnv-v0'

In [ ]:
class Normalizer():

    def __init__(self, nb_inputs):
        self.n = np.zeros(nb_inputs)
        self.mean = np.zeros(nb_inputs)
        self.mean_diff = np.zeros(nb_inputs)
        self.var = np.zeros(nb_inputs)

    def observe(self, x):
        self.n += 1.
        last_mean = self.mean.copy()
        self.mean += (x - self.mean) / self.n
        self.mean_diff += (x - last_mean) * (x - self.mean)
        self.var = (self.mean_diff / self.n).clip(min=1e-2)

    def normalize(self, inputs):
        obs_mean = self.mean
        obs_std = np.sqrt(self.var)
        return (inputs - obs_mean) / obs_std

In [ ]:
class Policy():

    def __init__(self, input_size, output_size, hp):
        self.theta = np.zeros((output_size, input_size))
        self.hp = hp

    def evaluate(self, input, delta=None, direction=None):
        if direction is None:
            return self.theta.dot(input)
        elif direction == "positive":
            return (self.theta + self.hp.noise * delta).dot(input)
        else:
            return (self.theta - self.hp.noise * delta).dot(input)

    def sample_deltas(self):
        return [np.random.randn(*self.theta.shape) for _ in range(self.hp.nb_directions)]

    def update(self, rollouts, sigma_r):
        step = np.zeros(self.theta.shape)
        for r_pos, r_neg, d in rollouts:
            step += (r_pos - r_neg) * d
        self.theta += self.hp.learning_rate / (self.hp.nb_best_directions * sigma_r) * step

In [ ]:
class Explorers():
    def __init__(self, env:StockTradingEnv, policy:Policy, normalizer:Normalizer, hp:Hp):
        self.normalizer = normalizer
        self.policy = policy
        self.env = env
        self.hp = hp
        
        
    def positive(self, delta=None):
        direction='positive'
        state = self.env.reset()
        done = False
        num_plays = 0.
        sum_rewards = 0
        while not done and num_plays < self.hp.episode_length:
            self.normalizer.observe(state)
            state = self.normalizer.normalize(state)
            action = self.policy.evaluate(state, delta, direction)
            state, reward, done, _ = self.env.step(action)
            reward = max(min(reward, 1), -1)
            sum_rewards += reward
            num_plays += 1
        return sum_rewards

    def negative(self, delta=None):
        direction='negative'
        state = self.env.reset()
        done = False
        num_plays = 0.
        sum_rewards = 0
        while not done and num_plays < self.hp.episode_length:
            self.normalizer.observe(state)
            state = self.normalizer.normalize(state)
            action = self.policy.evaluate(state, delta, direction)
            state, reward, done, _ = self.env.step(action)
            reward = max(min(reward, 1), -1)
            sum_rewards += reward
            num_plays += 1
        return sum_rewards
    
    
def explore2(env, normalizer, policy, direction=None, delta=None):
    state = env.reset(locked=True)
    done = False
    num_plays = 0.
    sum_rewards = 0
    while not done and num_plays < hp.episode_length2:
        normalizer.observe(state)
        state = normalizer.normalize(state)
        action = policy.evaluate(state, delta, direction)
        state, reward, done, saldo = env.step(action)
        reward = max(min(reward, 1), -1)
        sum_rewards += round(reward,2)
        num_plays += 1
    return round(sum_rewards,2), round(saldo,2)

In [ ]:
# Training the AI

def train(env:StockTradingEnv, policy:Policy, normalizer:Normalizer, hp:Hp):
    n_workers = os.cpu_count()



    for step in range(hp.nb_steps):

        # Initializing the perturbations deltas and the positive/negative rewards
        deltas = policy.sample_deltas()
        positive_rewards = [0] * hp.nb_directions
        negative_rewards = [0] * hp.nb_directions
        explorers = Explorers(env, policy, normalizer, hp)
        # Getting the positive rewards in the positive directions
        #t1_start = perf_counter()
        #for k in range(hp.nb_directions):
        #    positive_rewards[k] = explorers.positive(delta=deltas[k])
        #    negative_rewards[k] = explorers.negative(delta=deltas[k])
        #positive_rewards = [df for df in results]
        #negative_rewards = [df for df in results]

        #t1_stop = perf_counter()
        #print("Elapsed time during the whole program in seconds:",t1_stop-t1_start)

        # Getting the negative rewards in the negative/opposite directions
        #for k in range(hp.nb_directions):
        #    negative_rewards[k] = explorers.negative(delta=deltas[k])

        #t1_start = perf_counter()

        #pool = Pool(processes=n_workers)
        #results = pool.map(explorers.positive, deltas)
        #positive_rewards = [df for df in results]

        #pool = Pool(processes=n_workers)
        #results = pool.map(explorers.negative, deltas)
        #negative_rewards = [df for df in results]
        #pool = None
        #results = None
        gc.collect()
        #t1_stop = perf_counter()
        #print("Elapsed time during the whole program in seconds:",t1_stop-t1_start)
    
        t1_start = perf_counter()
        with cf.ProcessPoolExecutor(max_workers=n_workers) as executor:
             results = executor.map(explorers.positive, deltas)
        t1_stop = perf_counter()
        # print("Elapsed time during the whole program in seconds:",t1_stop-t1_start)


        positive_rewards = [df for df in results]

        with cf.ProcessPoolExecutor(max_workers=n_workers) as executor:
            results = executor.map(explorers.negative, deltas)
        negative_rewards = [df for df in results]
    
        # Gathering all the positive/negative rewards to compute the standard deviation of these rewards
        all_rewards = np.array(positive_rewards + negative_rewards)
        sigma_r = all_rewards.std()

        # Sorting the rollouts by the max(r_pos, r_neg) and selecting the best directions
        scores = {k: max(r_pos, r_neg) for k, (r_pos, r_neg) in enumerate(zip(positive_rewards, negative_rewards))}
        order = sorted(scores.keys(), key=lambda x: scores[x], reverse=True)[:hp.nb_best_directions]
        rollouts = [(positive_rewards[k], negative_rewards[k], deltas[k]) for k in order]

        # Updating our policy
        policy.update(rollouts, sigma_r)

        # Printing the final reward of the policy after the update
        #if step % 100==0:
        reward_evaluation, saldo = explore2(env, normalizer, policy)
        print('Step:', step, 'Reward:', reward_evaluation, 'Saldo', saldo)
        for i in range(0, len(history)-1):
            if 'BTC' in history[i][0]:
                history[i][1][:10000].to_csv(f'results/{int(round(saldo, 0))}_{int(round(reward_evaluation, 0))}_.csv', sep=';', decimal=',')
                pickle.dump( policy.theta, open(f'pickle/{int(round(saldo, 0))}_{int(round(reward_evaluation, 0))}_.pkl', "wb" ) )
#         else:
#             if step % 10==0:
#                 print('Step:', step)

In [ ]:
if __name__=="__main__":
    hp = Hp()
    np.random.seed(hp.seed)
    env = StockTradingEnv(df)
    nb_inputs = env.observation_space
    nb_outputs = env.action_space
    policy = Policy(nb_inputs, nb_outputs, hp)
    normalizer = Normalizer(nb_inputs)
    train(env, policy, normalizer, hp)